In [5]:
import pandas as pd
import re
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import re
import spacy
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset, load_from_disk
import accelerate

Model building

In [17]:
# Load the pre-trained T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Define a function to tokenize the 'article' and 'highlights' columns
def tokenize_function(examples):
    # Tokenize the 'article' text with a maximum length of 512 tokens, truncating longer sequences
    model_inputs = tokenizer(examples['article'], max_length=512, truncation=True)
    
    # Tokenize the 'highlights' text (used as labels) with a maximum length of 150 tokens
    labels = tokenizer(examples['highlights'], max_length=150, truncation=True)
    
    # Set the 'input_ids' from the tokenized highlights as labels for the model
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

# Apply the tokenization function to the training, validation, and test datasets
# The map function applies the tokenization in batches for efficiency
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
# !pip install accelerate -U

In [33]:
import accelerate
print(accelerate.__version__)

0.33.0


In [5]:
# pip install accelerate transformers torch datasets

In [1]:
from datasets import load_from_disk

# Load the pre-tokenized datasets from disk
# The datasets are stored in a specified directory and are loaded into Dataset objects
tokenized_train_dataset = load_from_disk('tokenized_datasets/train')
tokenized_val_dataset = load_from_disk('tokenized_datasets/val')
tokenized_test_dataset = load_from_disk('tokenized_datasets/test')

Adjusting Padding for Tokenized Text Data in T5 Model Training

In [35]:
import torch
from transformers import T5Tokenizer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)

# Function to adjust padding
def adjust_padding(examples, max_length=512):
    # Adjust inputs
    inputs = tokenizer.pad(
        {"input_ids": examples["input_ids"], "attention_mask": examples["attention_mask"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Adjust labels
    labels = tokenizer.pad(
        {"input_ids": examples["labels"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Convert tensors to lists
    examples["input_ids"] = inputs["input_ids"].tolist()
    examples["attention_mask"] = inputs["attention_mask"].tolist()
    examples["labels"] = labels["input_ids"].tolist()
    return examples

# Adjust padding for each dataset
max_length_article = 512
max_length_summary = 150

tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_test_dataset = tokenized_test_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Shuffling, Subsampling, and Padding Adjustments for T5 Model Training on Smaller Datasets

In [1]:
# subsampling the dataset before adjusting the padding because this approach took too long as the data is huge
'''from datasets import load_from_disk
import torch
from transformers import T5Tokenizer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)

# Load the tokenized datasets
tokenized_train_dataset = load_from_disk('tokenized_datasets/train')
tokenized_val_dataset = load_from_disk('tokenized_datasets/val')
tokenized_test_dataset = load_from_disk('tokenized_datasets/test')

# Function to adjust padding
def adjust_padding(examples, max_length=512):
    # Adjust inputs
    inputs = tokenizer.pad(
        {"input_ids": examples["input_ids"], "attention_mask": examples["attention_mask"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Adjust labels
    labels = tokenizer.pad(
        {"input_ids": examples["labels"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Convert tensors to lists
    examples["input_ids"] = inputs["input_ids"].tolist()
    examples["attention_mask"] = inputs["attention_mask"].tolist()
    examples["labels"] = labels["input_ids"].tolist()
    return examples

# Adjust padding for each dataset
max_length_article = 512
max_length_summary = 150

tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_test_dataset = tokenized_test_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
'''

'from datasets import load_from_disk\nimport torch\nfrom transformers import T5Tokenizer\n\n# Load the tokenizer\ntokenizer = T5Tokenizer.from_pretrained(\'t5-small\', legacy=False)\n\n# Load the tokenized datasets\ntokenized_train_dataset = load_from_disk(\'tokenized_datasets/train\')\ntokenized_val_dataset = load_from_disk(\'tokenized_datasets/val\')\ntokenized_test_dataset = load_from_disk(\'tokenized_datasets/test\')\n\n# Function to adjust padding\ndef adjust_padding(examples, max_length=512):\n    # Adjust inputs\n    inputs = tokenizer.pad(\n        {"input_ids": examples["input_ids"], "attention_mask": examples["attention_mask"]},\n        padding="max_length",\n        max_length=max_length,\n        return_tensors="pt"\n    )\n    # Adjust labels\n    labels = tokenizer.pad(\n        {"input_ids": examples["labels"]},\n        padding="max_length",\n        max_length=max_length,\n        return_tensors="pt"\n    )\n    # Convert tensors to lists\n    examples["input_ids"] = 

In [1]:
from datasets import load_from_disk
import random
from tqdm import tqdm
from transformers import T5Tokenizer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)

# Load the tokenized datasets
tokenized_train_dataset = load_from_disk('tokenized_datasets/train')
tokenized_val_dataset = load_from_disk('tokenized_datasets/val')
tokenized_test_dataset = load_from_disk('tokenized_datasets/test')

# Function to shuffle and select a subset of the dataset with progress bar
def shuffle_and_select(dataset, num_samples, seed=42):
    indices = list(range(len(dataset)))
    random.seed(seed)
    random.shuffle(indices)
    selected_indices = indices[:num_samples]
    subset = dataset.select(selected_indices)
    return subset

# Select a smaller subset of the dataset with progress bar
print("Shuffling and selecting smaller train dataset...")
small_train_dataset = shuffle_and_select(tokenized_train_dataset, 5000)
print("Shuffling and selecting smaller val dataset...")
small_val_dataset = shuffle_and_select(tokenized_val_dataset, 1000)

# Adjust padding for smaller datasets
def adjust_padding(examples, max_length=512):
    # Adjust inputs
    inputs = tokenizer.pad(
        {"input_ids": examples["input_ids"], "attention_mask": examples["attention_mask"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Adjust labels
    labels = tokenizer.pad(
        {"input_ids": examples["labels"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Convert tensors to lists
    examples["input_ids"] = inputs["input_ids"].tolist()
    examples["attention_mask"] = inputs["attention_mask"].tolist()
    examples["labels"] = labels["input_ids"].tolist()
    return examples

print("Adjusting padding for smaller train dataset...")
small_train_dataset = small_train_dataset.map(lambda examples: adjust_padding(examples, max_length=512), batched=True)
print("Adjusting padding for smaller val dataset...")
small_val_dataset = small_val_dataset.map(lambda examples: adjust_padding(examples, max_length=512), batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Shuffling and selecting smaller train dataset...
Shuffling and selecting smaller val dataset...
Adjusting padding for smaller train dataset...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Adjusting padding for smaller val dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
# Tokenize the 'article' and 'highlights' columns from the dataset.
# tokenize text data and prepare it for input into a T5 model
def tokenize_function(examples):
    model_inputs = tokenizer(examples['article'], max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(examples['highlights'], max_length=150, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

Model Training

In [5]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Adjust training parameters
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduce the number of epochs
    per_device_train_batch_size=8,  # Increase the batch size if you have enough GPU memory
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    fp16=True,  # Use mixed precision training if supported by your hardware
    disable_tqdm=False,  # Ensure tqdm progress bar is enabled
    report_to="none"  # Ensure no integration with external logging services
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
)

# Show progress with tqdm
print("Starting training...")
trainer.train()
print("Training completed.")

# Save the model and tokenizer
model.save_pretrained('saved_model/t5-small')
tokenizer.save_pretrained('saved_model/t5-small')




Starting training...


Epoch,Training Loss,Validation Loss
1,0.335300,0.321402


Training completed.


('saved_model/t5-small\\tokenizer_config.json',
 'saved_model/t5-small\\special_tokens_map.json',
 'saved_model/t5-small\\spiece.model',
 'saved_model/t5-small\\added_tokens.json')